In [1]:
import pandas as pd
import numpy as np
import tqdm
import warnings
import matplotlib.pyplot as plt
from sklearn.metrics import roc_auc_score

pd.set_option("max_rows", 200)
pd.set_option("max_columns", 50)
tqdm.tqdm.pandas()
%matplotlib inline

In [4]:
df = pd.read_pickle("../input/riiid-test-answer-prediction/split10/train_0.pickle").sort_values(["user_id", "timestamp"]).reset_index(drop=True)

In [5]:
import sys
sys.path.append("../")
# df = pd.read_pickle("../input/riiid-test-answer-prediction/split10/train_0.pickle")
df = df.sort_values(["user_id", "timestamp"])
# large_user_id = df["user_id"].value_counts()
# large_user_id = large_user_id[large_user_id > 1000] 
# df = df[df["user_id"].isin(large_user_id.index)]
# df = df[df["answered_correctly"] != -1]

In [6]:
df["answered_correctly"] = df["answered_correctly"].replace(-1, np.nan)

In [7]:
def f(series):
    return series.shift(1).cumsum().fillna(0) / np.arange(len(series))
df["target_enc_user_id"] = df.groupby("user_id")["answered_correctly"].transform(f)

In [11]:
aa = df.groupby(["content_id", "user_answer"]).size() / df.groupby("content_id").size()

In [17]:
df = pd.merge(df, aa.reset_index().rename(columns={0: "ratio"}))

In [21]:
df["ratio_bin"] = pd.cut(df["ratio"], [-1, 0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9, 1.01])

In [23]:
df.groupby("ratio_bin")["target_enc_user_id"].describe()

,count,mean,std,min,25%,50%,75%,max
ratio_bin,,,,,,,,
"(-1.0, 0.1]",722410.0,0.568095,0.154991,0.0,0.500000,0.598592,0.667205,1.0
"(0.1, 0.2]",1101096.0,0.586376,0.157236,0.0,0.521429,0.617397,0.683473,1.0
"(0.2, 0.3]",853527.0,0.591752,0.163197,0.0,0.525424,0.625000,0.693878,1.0
"(0.3, 0.4]",629160.0,0.611893,0.168119,0.0,0.550523,0.642857,0.711111,1.0
"(0.4, 0.5]",700893.0,0.613097,0.173599,0.0,0.555556,0.649275,0.716237,1.0
"(0.5, 0.6]",958369.0,0.623948,0.163672,0.0,0.577465,0.657754,0.719608,1.0
"(0.6, 0.7]",1364772.0,0.632540,0.151396,0.0,0.587682,0.660104,0.719008,1.0
"(0.7, 0.8]",1661611.0,0.635468,0.137701,0.0,0.588037,0.656724,0.715026,1.0
"(0.8, 0.9]",1339830.0,0.638650,0.130218,0.0,0.586207,0.656573,0.715880,1.0
